In [157]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

from utils.portfolio import PortfolioManagement

from environments.environment_old import Environment


config = {}
## setting for a toy dataset
config['lag'] = 5
config['epochs'] = 10
config['epi_sz'] = 60   # epi_sz is equal to the number of possible time steps in one epidoe.
config['total_no_epi'] = 20  # floor(91057/1460), 91057 == # rows of any currency.
config['num_batches'] = 5
config['batch_sz'] = 8
#######################
## setting for a real dataset
# config['lag'] = 5
# config['epochs'] = 30
# config['epi_sz'] = 20   # epi_sz is equal to the number of possible time steps in one epidoe.
# config['total_no_epi'] = 30  # floor(91057/1460), 91057 == # rows of any currency.
# config['num_batches'] = 10
# config['batch_sz'] = 16
#######################
config['state_sz'] = 9*config['lag']   # state = [time step, 9*lag log returns]
config['split_sz'] = config['epi_sz'] + config['lag']
config['lr_rate'] = 0.001
config['momentum'] = 0.9
config['decay_rate'] = 0.03
config['bins'] = np.concatenate([[-10], np.arange(-1e-4, 1.1e-4, 1e-5), [10]])
config['channels'] = 30
config['buffer_sz'] = 50000

assert config['epi_sz'] > config['batch_sz']

# config['start_learning'] = 5000
config['action_set'] = [-1, 0, 1]
config['M'] = 1     # The number of the target currency pairs to invest in.
# config['N'] = 23    # The number of bins in a market-image matrix
# config['C'] = 30    # The number of channels in a market-image matrix
config['obs_sz'] = 183

X_train = pd.read_csv('../dataset/X_train.csv', sep=',')
X_test = pd.read_csv('../dataset/X_test.csv', sep=',')
assert config['total_no_epi']*config['split_sz'] <= np.min([X_train[key].shape[0] for key in X_train.keys()]), "Training set has less data points than # of episodes * # of splits."

In [158]:
X_train = X_train[X_train.ccy=='gbpusd']
X_train_close = X_train.close.values
X_test = X_test[X_test.ccy=='gbpusd']
X_test_close = X_test.close.values
max_epi_no = int(X_test.shape[0]/((config['epi_sz']/2)*config['lag']*9))

In [159]:
config['target_currency'] = 'gbpusd'

In [160]:
env = Environment(X_test, config)

In [161]:
def get_action(rate, std, mean):
    threshold = mean + std
    if rate > threshold:
        return -1
    elif rate < threshold:
        return 1
    return 0

In [162]:
def run(mode='train', epoch=1):
    # We use `decay_step` and `decay_rate` when we do a variant of the e-greedy strategy.
    decay_step = 0
    epi_rewards = []
    epi_losses = []
    epi_val_losses = []
    epi_actions = []
    epi_pf_values = []

    for epi_idx, epi_no in enumerate(np.random.choice(max_epi_no, size=30)):
        t, obs, price_tuple, is_finished = env.reset(epi_no)  # epi_no a random index
        price, next_price = price_tuple

        mean = np.average(price[:30])
        std = np.std(price[:30])

        for i, rate in enumerate(price):
            if i == 0:
                start_rate = rate
                step_rewards = np.empty(shape=(0))
                step_losses = np.empty(shape=(0))
                step_val_losses = np.empty(shape=(0))
                step_actions = np.empty(shape=(0), dtype=np.int)
                step_pf_values = np.empty(shape=(0))

                # cash price is always 1. So the initial price vector is [1, price[0]].
                price_vector = np.array([1, price[0]])

                # M = 1 because our FX to trade is just one currency pair(the target currrency) at this moment
                pf = PortfolioManagement(init_price=price_vector, M=1)
                continue
            elif i < 30:
                continue


            action = get_action(rate=rate, std=std, mean=mean)

            target_wt = pf.get_target_weight(action)

            # Do this trade
            # In this pf.trade() method, we do:
            #   append self.wt, self.qty, self.price, self.rel_price into the corresponding lists.
            pf.trade(cur_wt=pf.wt[-1], target_wt=target_wt, target_price=next_price[0])

            reward = pf.get_pf_value(-1) - pf.get_pf_value(-2)   # getting a reward function will be implemented within self.env.step() later.

            step_rewards = np.append(step_rewards, reward)
            step_actions = np.append(step_actions, action)
            step_pf_values = np.append(step_pf_values, pf.get_pf_value())

            print('Episode/Time step {:2d}/{:2d}. Reward {:.2f}'.format(epi_idx+1, i+1, reward))
        ### end of trading ###

        epi_rewards.append(np.sum(step_rewards))
        epi_actions.append(step_actions)
        epi_pf_values.append(step_pf_values)

    results_dict = {
        'epi_rewards': epi_rewards,
        'epi_losses': epi_losses,
        'epi_val_losses': epi_val_losses,
        'epi_actions': epi_actions,
        'epi_pf_values': epi_pf_values
    }

    return results_dict


In [163]:
run()

Episode/Time step  1/31. Reward -0.89
Episode/Time step  1/32. Reward 0.00
Episode/Time step  1/33. Reward -0.22
Episode/Time step  1/34. Reward -1.00
Episode/Time step  1/35. Reward -2.00
Episode/Time step  1/36. Reward -1.39
Episode/Time step  1/37. Reward -0.62
Episode/Time step  1/38. Reward 0.00
Episode/Time step  1/39. Reward 0.00
Episode/Time step  1/40. Reward 0.00
Episode/Time step  1/41. Reward 0.00
Episode/Time step  1/42. Reward 0.00
Episode/Time step  1/43. Reward 0.00
Episode/Time step  1/44. Reward 0.00
Episode/Time step  1/45. Reward 0.00
Episode/Time step  1/46. Reward 0.00
Episode/Time step  1/47. Reward 0.00
Episode/Time step  1/48. Reward 0.00
Episode/Time step  1/49. Reward 0.00
Episode/Time step  1/50. Reward -0.77
Episode/Time step  1/51. Reward -1.00
Episode/Time step  1/52. Reward -0.85
Episode/Time step  1/53. Reward 0.00
Episode/Time step  1/54. Reward 0.00
Episode/Time step  1/55. Reward 0.00
Episode/Time step  1/56. Reward -1.85
Episode/Time step  1/57. Rew

Episode/Time step  8/54. Reward 0.00
Episode/Time step  8/55. Reward 0.00
Episode/Time step  8/56. Reward 0.00
Episode/Time step  8/57. Reward 0.00
Episode/Time step  8/58. Reward 0.00
Episode/Time step  8/59. Reward -0.69
Episode/Time step  8/60. Reward -0.62
Episode/Time step  9/31. Reward -0.33
Episode/Time step  9/32. Reward 0.00
Episode/Time step  9/33. Reward 0.00
Episode/Time step  9/34. Reward 0.00
Episode/Time step  9/35. Reward 0.00
Episode/Time step  9/36. Reward 0.00
Episode/Time step  9/37. Reward 0.00
Episode/Time step  9/38. Reward 0.00
Episode/Time step  9/39. Reward 0.00
Episode/Time step  9/40. Reward 0.00
Episode/Time step  9/41. Reward 0.00
Episode/Time step  9/42. Reward 0.00
Episode/Time step  9/43. Reward 0.00
Episode/Time step  9/44. Reward 0.00
Episode/Time step  9/45. Reward 0.00
Episode/Time step  9/46. Reward 0.00
Episode/Time step  9/47. Reward 0.00
Episode/Time step  9/48. Reward 0.00
Episode/Time step  9/49. Reward 0.00
Episode/Time step  9/50. Reward 0.0

Episode/Time step 23/31. Reward -0.45
Episode/Time step 23/32. Reward -1.00
Episode/Time step 23/33. Reward -0.70
Episode/Time step 23/34. Reward -2.00
Episode/Time step 23/35. Reward -1.39
Episode/Time step 23/36. Reward -1.70
Episode/Time step 23/37. Reward -1.70
Episode/Time step 23/38. Reward -0.39
Episode/Time step 23/39. Reward -1.00
Episode/Time step 23/40. Reward -0.70
Episode/Time step 23/41. Reward -2.00
Episode/Time step 23/42. Reward -1.39
Episode/Time step 23/43. Reward -0.70
Episode/Time step 23/44. Reward -2.00
Episode/Time step 23/45. Reward -1.39
Episode/Time step 23/46. Reward -0.70
Episode/Time step 23/47. Reward -2.24
Episode/Time step 23/48. Reward -1.30
Episode/Time step 23/49. Reward -0.30
Episode/Time step 23/50. Reward 0.00
Episode/Time step 23/51. Reward 0.00
Episode/Time step 23/52. Reward 0.00
Episode/Time step 23/53. Reward -2.23
Episode/Time step 23/54. Reward -0.38
Episode/Time step 23/55. Reward 0.00
Episode/Time step 23/56. Reward 0.00
Episode/Time step

{'epi_rewards': [-12.648450000000594,
  -0.32652999999845633,
  -1.0178200000009383,
  -1.2921599999990576,
  -4.461280000001352,
  -10.547450000000026,
  -0.5563600000004953,
  -4.461280000001352,
  -0.32652999999845633,
  -12.648450000000594,
  -4.461280000001352,
  -32.07104000000072,
  -5.700749999999971,
  -21.366400000000795,
  -5.607719999999972,
  -5.700749999999971,
  -8.871780000001309,
  -29.969380000000456,
  -4.461280000001352,
  -34.63774999999987,
  -12.648450000000594,
  -34.63774999999987,
  -25.64494999999988,
  -25.64494999999988,
  -34.63774999999987,
  -5.607719999999972,
  -25.64494999999988,
  -8.871780000001309,
  -21.366400000000795,
  -21.366400000000795],
 'epi_losses': [],
 'epi_val_losses': [],
 'epi_actions': [array([ 1,  1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]),
  

In [164]:
mr_results_dict = {}
for epoch in range(1,21):
    mr_results_dict[epoch] = run()

Episode/Time step  1/31. Reward -0.84
Episode/Time step  1/32. Reward -0.70
Episode/Time step  1/33. Reward -2.00
Episode/Time step  1/34. Reward -1.39
Episode/Time step  1/35. Reward -0.70
Episode/Time step  1/36. Reward -2.00
Episode/Time step  1/37. Reward -1.39
Episode/Time step  1/38. Reward -1.70
Episode/Time step  1/39. Reward -1.70
Episode/Time step  1/40. Reward -0.39
Episode/Time step  1/41. Reward -1.00
Episode/Time step  1/42. Reward -0.70
Episode/Time step  1/43. Reward -1.00
Episode/Time step  1/44. Reward -0.70
Episode/Time step  1/45. Reward -0.70
Episode/Time step  1/46. Reward -1.00
Episode/Time step  1/47. Reward -0.70
Episode/Time step  1/48. Reward -2.00
Episode/Time step  1/49. Reward -1.39
Episode/Time step  1/50. Reward -0.70
Episode/Time step  1/51. Reward -0.70
Episode/Time step  1/52. Reward -1.00
Episode/Time step  1/53. Reward -0.70
Episode/Time step  1/54. Reward -2.00
Episode/Time step  1/55. Reward -1.39
Episode/Time step  1/56. Reward -0.70
Episode/Time

Episode/Time step 14/31. Reward -1.28
Episode/Time step 14/32. Reward -1.69
Episode/Time step 14/33. Reward -0.39
Episode/Time step 14/34. Reward -1.00
Episode/Time step 14/35. Reward -0.69
Episode/Time step 14/36. Reward -2.00
Episode/Time step 14/37. Reward -1.39
Episode/Time step 14/38. Reward -0.69
Episode/Time step 14/39. Reward -2.00
Episode/Time step 14/40. Reward -1.39
Episode/Time step 14/41. Reward -1.69
Episode/Time step 14/42. Reward -1.69
Episode/Time step 14/43. Reward -0.39
Episode/Time step 14/44. Reward -1.00
Episode/Time step 14/45. Reward -0.69
Episode/Time step 14/46. Reward -1.00
Episode/Time step 14/47. Reward -0.69
Episode/Time step 14/48. Reward -0.69
Episode/Time step 14/49. Reward -1.00
Episode/Time step 14/50. Reward -0.69
Episode/Time step 14/51. Reward -2.00
Episode/Time step 14/52. Reward -1.39
Episode/Time step 14/53. Reward -0.69
Episode/Time step 14/54. Reward -0.69
Episode/Time step 14/55. Reward -1.00
Episode/Time step 14/56. Reward -0.69
Episode/Time

Episode/Time step 23/57. Reward -1.00
Episode/Time step 23/58. Reward -0.69
Episode/Time step 23/59. Reward -1.00
Episode/Time step 23/60. Reward -0.69
Episode/Time step 24/31. Reward -1.00
Episode/Time step 24/32. Reward -0.60
Episode/Time step 24/33. Reward 0.00
Episode/Time step 24/34. Reward 0.00
Episode/Time step 24/35. Reward -0.78
Episode/Time step 24/36. Reward -0.53
Episode/Time step 24/37. Reward 0.00
Episode/Time step 24/38. Reward 0.00
Episode/Time step 24/39. Reward 0.00
Episode/Time step 24/40. Reward 0.00
Episode/Time step 24/41. Reward 0.00
Episode/Time step 24/42. Reward 0.00
Episode/Time step 24/43. Reward 0.00
Episode/Time step 24/44. Reward 0.00
Episode/Time step 24/45. Reward 0.00
Episode/Time step 24/46. Reward 0.00
Episode/Time step 24/47. Reward 0.00
Episode/Time step 24/48. Reward 0.00
Episode/Time step 24/49. Reward 0.00
Episode/Time step 24/50. Reward 0.00
Episode/Time step 24/51. Reward 0.00
Episode/Time step 24/52. Reward -1.47
Episode/Time step 24/53. Rewa

Episode/Time step  4/54. Reward 0.00
Episode/Time step  4/55. Reward 0.00
Episode/Time step  4/56. Reward 0.00
Episode/Time step  4/57. Reward 0.00
Episode/Time step  4/58. Reward 0.00
Episode/Time step  4/59. Reward 0.00
Episode/Time step  4/60. Reward 0.00
Episode/Time step  5/31. Reward -0.56
Episode/Time step  5/32. Reward 0.00
Episode/Time step  5/33. Reward 0.00
Episode/Time step  5/34. Reward 0.00
Episode/Time step  5/35. Reward 0.00
Episode/Time step  5/36. Reward 0.00
Episode/Time step  5/37. Reward 0.00
Episode/Time step  5/38. Reward 0.00
Episode/Time step  5/39. Reward 0.00
Episode/Time step  5/40. Reward 0.00
Episode/Time step  5/41. Reward 0.00
Episode/Time step  5/42. Reward 0.00
Episode/Time step  5/43. Reward 0.00
Episode/Time step  5/44. Reward 0.00
Episode/Time step  5/45. Reward 0.00
Episode/Time step  5/46. Reward 0.00
Episode/Time step  5/47. Reward 0.00
Episode/Time step  5/48. Reward 0.00
Episode/Time step  5/49. Reward 0.00
Episode/Time step  5/50. Reward 0.00


Episode/Time step 13/60. Reward 0.00
Episode/Time step 14/31. Reward -0.83
Episode/Time step 14/32. Reward 0.00
Episode/Time step 14/33. Reward 0.00
Episode/Time step 14/34. Reward 0.00
Episode/Time step 14/35. Reward 0.00
Episode/Time step 14/36. Reward 0.00
Episode/Time step 14/37. Reward 0.00
Episode/Time step 14/38. Reward 0.00
Episode/Time step 14/39. Reward 0.00
Episode/Time step 14/40. Reward 0.00
Episode/Time step 14/41. Reward 0.00
Episode/Time step 14/42. Reward 0.00
Episode/Time step 14/43. Reward -0.35
Episode/Time step 14/44. Reward -1.00
Episode/Time step 14/45. Reward -0.69
Episode/Time step 14/46. Reward -1.00
Episode/Time step 14/47. Reward -0.69
Episode/Time step 14/48. Reward -2.00
Episode/Time step 14/49. Reward -1.39
Episode/Time step 14/50. Reward -0.69
Episode/Time step 14/51. Reward -0.03
Episode/Time step 14/52. Reward 0.00
Episode/Time step 14/53. Reward 0.00
Episode/Time step 14/54. Reward 0.00
Episode/Time step 14/55. Reward 0.00
Episode/Time step 14/56. Rew

...

Episode/Time step 19/59. Reward 0.00
Episode/Time step 19/60. Reward 0.00
Episode/Time step 20/31. Reward -0.59
Episode/Time step 20/32. Reward 0.00
Episode/Time step 20/33. Reward -0.83
Episode/Time step 20/34. Reward -0.48
Episode/Time step 20/35. Reward 0.00
Episode/Time step 20/36. Reward 0.00
Episode/Time step 20/37. Reward -0.21
Episode/Time step 20/38. Reward -1.00
Episode/Time step 20/39. Reward -2.00
Episode/Time step 20/40. Reward -0.71
Episode/Time step 20/41. Reward 0.00
Episode/Time step 20/42. Reward 0.00
Episode/Time step 20/43. Reward 0.00
Episode/Time step 20/44. Reward 0.00
Episode/Time step 20/45. Reward 0.00
Episode/Time step 20/46. Reward 0.00
Episode/Time step 20/47. Reward 0.00
Episode/Time step 20/48. Reward 0.00
Episode/Time step 20/49. Reward 0.00
Episode/Time step 20/50. Reward 0.00
Episode/Time step 20/51. Reward 0.00
Episode/Time step 20/52. Reward 0.00
Episode/Time step 20/53. Reward 0.00
Episode/Time step 20/54. Reward 0.00
Episode/Time step 20/55. Reward

Episode/Time step 30/31. Reward -0.63
Episode/Time step 30/32. Reward 0.00
Episode/Time step 30/33. Reward 0.00
Episode/Time step 30/34. Reward 0.00
Episode/Time step 30/35. Reward -0.74
Episode/Time step 30/36. Reward -1.00
Episode/Time step 30/37. Reward -0.70
Episode/Time step 30/38. Reward -2.00
Episode/Time step 30/39. Reward -1.39
Episode/Time step 30/40. Reward -0.70
Episode/Time step 30/41. Reward -0.70
Episode/Time step 30/42. Reward -1.00
Episode/Time step 30/43. Reward -0.70
Episode/Time step 30/44. Reward -2.00
Episode/Time step 30/45. Reward -1.39
Episode/Time step 30/46. Reward -0.70
Episode/Time step 30/47. Reward -2.00
Episode/Time step 30/48. Reward -1.39
Episode/Time step 30/49. Reward -1.70
Episode/Time step 30/50. Reward -1.70
Episode/Time step 30/51. Reward -0.39
Episode/Time step 30/52. Reward -1.00
Episode/Time step 30/53. Reward -0.70
Episode/Time step 30/54. Reward -1.00
Episode/Time step 30/55. Reward -0.70
Episode/Time step 30/56. Reward -0.70
Episode/Time st

Episode/Time step  9/31. Reward -0.42
Episode/Time step  9/32. Reward -1.00
Episode/Time step  9/33. Reward -0.69
Episode/Time step  9/34. Reward -2.00
Episode/Time step  9/35. Reward -1.39
Episode/Time step  9/36. Reward -1.69
Episode/Time step  9/37. Reward -1.69
Episode/Time step  9/38. Reward -0.39
Episode/Time step  9/39. Reward -1.00
Episode/Time step  9/40. Reward -0.69
Episode/Time step  9/41. Reward -1.00
Episode/Time step  9/42. Reward -0.69
Episode/Time step  9/43. Reward -0.69
Episode/Time step  9/44. Reward -1.00
Episode/Time step  9/45. Reward -0.69
Episode/Time step  9/46. Reward -2.00
Episode/Time step  9/47. Reward -1.39
Episode/Time step  9/48. Reward -0.69
Episode/Time step  9/49. Reward -2.00
Episode/Time step  9/50. Reward -1.39
Episode/Time step  9/51. Reward -1.69
Episode/Time step  9/52. Reward -1.69
Episode/Time step  9/53. Reward -0.39
Episode/Time step  9/54. Reward -1.00
Episode/Time step  9/55. Reward -0.69
Episode/Time step  9/56. Reward -1.00
Episode/Time

Episode/Time step 18/31. Reward -0.34
Episode/Time step 18/32. Reward 0.00
Episode/Time step 18/33. Reward 0.00
Episode/Time step 18/34. Reward 0.00
Episode/Time step 18/35. Reward 0.00
Episode/Time step 18/36. Reward 0.00
Episode/Time step 18/37. Reward 0.00
Episode/Time step 18/38. Reward 0.00
Episode/Time step 18/39. Reward 0.00
Episode/Time step 18/40. Reward 0.00
Episode/Time step 18/41. Reward 0.00
Episode/Time step 18/42. Reward 0.00
Episode/Time step 18/43. Reward 0.00
Episode/Time step 18/44. Reward -0.33
Episode/Time step 18/45. Reward -1.00
Episode/Time step 18/46. Reward -0.69
Episode/Time step 18/47. Reward -1.00
Episode/Time step 18/48. Reward -0.69
Episode/Time step 18/49. Reward -2.00
Episode/Time step 18/50. Reward -1.39
Episode/Time step 18/51. Reward -0.69
Episode/Time step 18/52. Reward -2.00
Episode/Time step 18/53. Reward -1.39
Episode/Time step 18/54. Reward -1.69
Episode/Time step 18/55. Reward -1.69
Episode/Time step 18/56. Reward -0.39
Episode/Time step 18/57.

Episode/Time step 29/35. Reward -0.70
Episode/Time step 29/36. Reward -2.00
Episode/Time step 29/37. Reward -1.39
Episode/Time step 29/38. Reward -0.70
Episode/Time step 29/39. Reward -0.70
Episode/Time step 29/40. Reward -1.00
Episode/Time step 29/41. Reward -0.70
Episode/Time step 29/42. Reward -2.00
Episode/Time step 29/43. Reward -1.39
Episode/Time step 29/44. Reward -0.70
Episode/Time step 29/45. Reward -2.00
Episode/Time step 29/46. Reward -1.39
Episode/Time step 29/47. Reward -1.70
Episode/Time step 29/48. Reward -1.70
Episode/Time step 29/49. Reward -0.39
Episode/Time step 29/50. Reward -1.00
Episode/Time step 29/51. Reward -0.70
Episode/Time step 29/52. Reward -1.00
Episode/Time step 29/53. Reward -0.70
Episode/Time step 29/54. Reward -0.70
Episode/Time step 29/55. Reward -1.00
Episode/Time step 29/56. Reward -0.70
Episode/Time step 29/57. Reward -2.00
Episode/Time step 29/58. Reward -1.39
Episode/Time step 29/59. Reward -0.70
Episode/Time step 29/60. Reward -0.70
Episode/Time

In [165]:
mr_results_dict

{1: {'epi_rewards': [-35.93058000000019,
   -32.07104000000072,
   -12.790800000000672,
   -5.607719999999972,
   -5.700749999999971,
   -12.790800000000672,
   -5.607719999999972,
   -4.461280000001352,
   -29.969380000000456,
   -32.07104000000072,
   -8.871780000001309,
   -4.461280000001352,
   -10.547450000000026,
   -34.63774999999987,
   -10.547450000000026,
   -1.1595399999987421,
   -29.969380000000456,
   -1.1595399999987421,
   -0.07985000000007858,
   -32.07104000000072,
   -35.93058000000019,
   -29.969380000000456,
   -18.688099999999395,
   -10.547450000000026,
   -5.607719999999972,
   -0.5563600000004953,
   -8.665960000000268,
   -5.607719999999972,
   -29.969380000000456,
   -29.896499999998923],
  'epi_losses': [],
  'epi_val_losses': [],
  'epi_actions': [array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
   array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 

In [167]:
def plot(data_x, title, filename, legend, ylabel='Cumulative data_x', xlabel='Epochs', color=None):
    cmap = ['red', 'blue', 'green', 'black', 'gray']

    # Plot L of class 0
    fig, ax = plt.subplots(1, 1, figsize=(10, 7.5))
    
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    # Provide tick lines across the plot to help your viewers trace along the axis ticks. 
    plt.grid(True, which='major', axis='y', ls='--', lw=0.5, c='k', alpha=0.3)
    
#    plt.tick_params(axis='both', which='both', bottom=True, top=True,
#                   labelbottom=True, left=True, right=True, labelleft=True)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    runs = legend

    plt.title(title, fontsize=16)
    for i in range(len(data_x)):
        clr = cmap[i] if color is None else color
        plt.plot(range(1,len(data_x[0])+1), data_x[i], lw=2.5, color = clr, label=runs[i])

    plt.legend(ncol=2)
    
    fig.savefig(filename+'.png', figsize=(12,8))
    plt.show()
    fig.clear()


In [168]:
rewards = np.array([np.average(mr_results_dict[i]['epi_rewards']) for i in range(1,21)])

In [169]:
filename = '../results/final/mean_reversion/mr_test'

In [170]:
np.save(filename, rewards)

In [171]:
env = Environment(X_train, config)
max_epi_no = int(X_train.shape[0]/((config['epi_sz']/2)*config['lag']*9))

In [172]:
mr_results_dict = {}
for epoch in range(1,21):
    mr_results_dict[epoch] = run()

Episode/Time step  1/31. Reward -0.42
Episode/Time step  1/32. Reward -0.42
Episode/Time step  1/33. Reward 0.00
Episode/Time step  1/34. Reward 0.00
Episode/Time step  1/35. Reward 0.00
Episode/Time step  1/36. Reward 0.00
Episode/Time step  1/37. Reward 0.00
Episode/Time step  1/38. Reward 0.00
Episode/Time step  1/39. Reward 0.00
Episode/Time step  1/40. Reward 0.00
Episode/Time step  1/41. Reward 0.00
Episode/Time step  1/42. Reward 0.00
Episode/Time step  1/43. Reward -1.58
Episode/Time step  1/44. Reward -0.93
Episode/Time step  1/45. Reward -0.31
Episode/Time step  1/46. Reward -0.95
Episode/Time step  1/47. Reward -0.80
Episode/Time step  1/48. Reward -1.00
Episode/Time step  1/49. Reward -0.71
Episode/Time step  1/50. Reward 0.00
Episode/Time step  1/51. Reward 0.00
Episode/Time step  1/52. Reward 0.00
Episode/Time step  1/53. Reward 0.00
Episode/Time step  1/54. Reward 0.00
Episode/Time step  1/55. Reward 0.00
Episode/Time step  1/56. Reward 0.00
Episode/Time step  1/57. Rewa

Episode/Time step 10/31. Reward -0.88
Episode/Time step 10/32. Reward 0.00
Episode/Time step 10/33. Reward 0.00
Episode/Time step 10/34. Reward 0.00
Episode/Time step 10/35. Reward 0.00
Episode/Time step 10/36. Reward 0.00
Episode/Time step 10/37. Reward 0.00
Episode/Time step 10/38. Reward 0.00
Episode/Time step 10/39. Reward 0.00
Episode/Time step 10/40. Reward -1.51
Episode/Time step 10/41. Reward -0.45
Episode/Time step 10/42. Reward -0.59
Episode/Time step 10/43. Reward 0.00
Episode/Time step 10/44. Reward -0.14
Episode/Time step 10/45. Reward -1.13
Episode/Time step 10/46. Reward 0.00
Episode/Time step 10/47. Reward 0.00
Episode/Time step 10/48. Reward 0.00
Episode/Time step 10/49. Reward 0.00
Episode/Time step 10/50. Reward 0.00
Episode/Time step 10/51. Reward 0.00
Episode/Time step 10/52. Reward 0.00
Episode/Time step 10/53. Reward 0.00
Episode/Time step 10/54. Reward 0.00
Episode/Time step 10/55. Reward 0.00
Episode/Time step 10/56. Reward 0.00
Episode/Time step 10/57. Reward 

Episode/Time step 20/31. Reward -0.76
Episode/Time step 20/32. Reward 0.00
Episode/Time step 20/33. Reward 0.00
Episode/Time step 20/34. Reward -0.48
Episode/Time step 20/35. Reward -1.00
Episode/Time step 20/36. Reward -0.75
Episode/Time step 20/37. Reward -1.00
Episode/Time step 20/38. Reward -0.75
Episode/Time step 20/39. Reward -0.75
Episode/Time step 20/40. Reward -1.00
Episode/Time step 20/41. Reward -0.75
Episode/Time step 20/42. Reward -0.75
Episode/Time step 20/43. Reward -1.00
Episode/Time step 20/44. Reward -0.75
Episode/Time step 20/45. Reward -0.75
Episode/Time step 20/46. Reward -1.00
Episode/Time step 20/47. Reward -0.75
Episode/Time step 20/48. Reward -0.75
Episode/Time step 20/49. Reward -1.35
Episode/Time step 20/50. Reward -1.39
Episode/Time step 20/51. Reward -1.75
Episode/Time step 20/52. Reward -0.49
Episode/Time step 20/53. Reward -1.00
Episode/Time step 20/54. Reward -0.65
Episode/Time step 20/55. Reward 0.00
Episode/Time step 20/56. Reward 0.00
Episode/Time ste

Episode/Time step 28/46. Reward 0.00
Episode/Time step 28/47. Reward 0.00
Episode/Time step 28/48. Reward -0.42
Episode/Time step 28/49. Reward -1.00
Episode/Time step 28/50. Reward -1.13
Episode/Time step 28/51. Reward 0.00
Episode/Time step 28/52. Reward -0.31
Episode/Time step 28/53. Reward -1.00
Episode/Time step 28/54. Reward -0.72
Episode/Time step 28/55. Reward -1.00
Episode/Time step 28/56. Reward -0.72
Episode/Time step 28/57. Reward -2.00
Episode/Time step 28/58. Reward -1.45
Episode/Time step 28/59. Reward -1.72
Episode/Time step 28/60. Reward -1.72
Episode/Time step 29/31. Reward -0.27
Episode/Time step 29/32. Reward 0.00
Episode/Time step 29/33. Reward 0.00
Episode/Time step 29/34. Reward 0.00
Episode/Time step 29/35. Reward 0.00
Episode/Time step 29/36. Reward 0.00
Episode/Time step 29/37. Reward 0.00
Episode/Time step 29/38. Reward 0.00
Episode/Time step 29/39. Reward 0.00
Episode/Time step 29/40. Reward 0.00
Episode/Time step 29/41. Reward 0.00
Episode/Time step 29/42. 

Episode/Time step  7/31. Reward -0.78
Episode/Time step  7/32. Reward 0.00
Episode/Time step  7/33. Reward 0.00
Episode/Time step  7/34. Reward 0.00
Episode/Time step  7/35. Reward 0.00
Episode/Time step  7/36. Reward 0.00
Episode/Time step  7/37. Reward 0.00
Episode/Time step  7/38. Reward 0.00
Episode/Time step  7/39. Reward 0.00
Episode/Time step  7/40. Reward 0.00
Episode/Time step  7/41. Reward 0.00
Episode/Time step  7/42. Reward 0.00
Episode/Time step  7/43. Reward 0.00
Episode/Time step  7/44. Reward 0.00
Episode/Time step  7/45. Reward 0.00
Episode/Time step  7/46. Reward 0.00
Episode/Time step  7/47. Reward 0.00
Episode/Time step  7/48. Reward 0.00
Episode/Time step  7/49. Reward 0.00
Episode/Time step  7/50. Reward 0.00
Episode/Time step  7/51. Reward 0.00
Episode/Time step  7/52. Reward 0.00
Episode/Time step  7/53. Reward 0.00
Episode/Time step  7/54. Reward 0.00
Episode/Time step  7/55. Reward 0.00
Episode/Time step  7/56. Reward 0.00
Episode/Time step  7/57. Reward 0.00


...

Episode/Time step 10/35. Reward 0.00
Episode/Time step 10/36. Reward 0.00
Episode/Time step 10/37. Reward 0.00
Episode/Time step 10/38. Reward 0.00
Episode/Time step 10/39. Reward 0.00
Episode/Time step 10/40. Reward 0.00
Episode/Time step 10/41. Reward 0.00
Episode/Time step 10/42. Reward 0.00
Episode/Time step 10/43. Reward 0.00
Episode/Time step 10/44. Reward 0.00
Episode/Time step 10/45. Reward 0.00
Episode/Time step 10/46. Reward 0.00
Episode/Time step 10/47. Reward 0.00
Episode/Time step 10/48. Reward 0.00
Episode/Time step 10/49. Reward 0.00
Episode/Time step 10/50. Reward 0.00
Episode/Time step 10/51. Reward 0.00
Episode/Time step 10/52. Reward 0.00
Episode/Time step 10/53. Reward 0.00
Episode/Time step 10/54. Reward 0.00
Episode/Time step 10/55. Reward 0.00
Episode/Time step 10/56. Reward 0.00
Episode/Time step 10/57. Reward 0.00
Episode/Time step 10/58. Reward 0.00
Episode/Time step 10/59. Reward 0.00
Episode/Time step 10/60. Reward 0.00
Episode/Time step 11/31. Reward -0.83


Episode/Time step 19/31. Reward -0.11
Episode/Time step 19/32. Reward 0.00
Episode/Time step 19/33. Reward 0.00
Episode/Time step 19/34. Reward -0.79
Episode/Time step 19/35. Reward -0.48
Episode/Time step 19/36. Reward 0.00
Episode/Time step 19/37. Reward -1.52
Episode/Time step 19/38. Reward -0.47
Episode/Time step 19/39. Reward -1.00
Episode/Time step 19/40. Reward -0.74
Episode/Time step 19/41. Reward -2.00
Episode/Time step 19/42. Reward -1.47
Episode/Time step 19/43. Reward -1.74
Episode/Time step 19/44. Reward -0.47
Episode/Time step 19/45. Reward -1.00
Episode/Time step 19/46. Reward -0.74
Episode/Time step 19/47. Reward -0.22
Episode/Time step 19/48. Reward 0.00
Episode/Time step 19/49. Reward 0.00
Episode/Time step 19/50. Reward -0.51
Episode/Time step 19/51. Reward -1.00
Episode/Time step 19/52. Reward -1.01
Episode/Time step 19/53. Reward 0.00
Episode/Time step 19/54. Reward 0.00
Episode/Time step 19/55. Reward 0.00
Episode/Time step 19/56. Reward 0.00
Episode/Time step 19/

Episode/Time step 28/31. Reward -0.88
Episode/Time step 28/32. Reward 0.00
Episode/Time step 28/33. Reward 0.00
Episode/Time step 28/34. Reward 0.00
Episode/Time step 28/35. Reward 0.00
Episode/Time step 28/36. Reward 0.00
Episode/Time step 28/37. Reward 0.00
Episode/Time step 28/38. Reward 0.00
Episode/Time step 28/39. Reward 0.00
Episode/Time step 28/40. Reward -1.51
Episode/Time step 28/41. Reward -0.45
Episode/Time step 28/42. Reward -0.59
Episode/Time step 28/43. Reward 0.00
Episode/Time step 28/44. Reward -0.14
Episode/Time step 28/45. Reward -1.13
Episode/Time step 28/46. Reward 0.00
Episode/Time step 28/47. Reward 0.00
Episode/Time step 28/48. Reward 0.00
Episode/Time step 28/49. Reward 0.00
Episode/Time step 28/50. Reward 0.00
Episode/Time step 28/51. Reward 0.00
Episode/Time step 28/52. Reward 0.00
Episode/Time step 28/53. Reward 0.00
Episode/Time step 28/54. Reward 0.00
Episode/Time step 28/55. Reward 0.00
Episode/Time step 28/56. Reward 0.00
Episode/Time step 28/57. Reward 

Episode/Time step 11/31. Reward -1.12
Episode/Time step 11/32. Reward 0.00
Episode/Time step 11/33. Reward 0.00
Episode/Time step 11/34. Reward 0.00
Episode/Time step 11/35. Reward 0.00
Episode/Time step 11/36. Reward 0.00
Episode/Time step 11/37. Reward 0.00
Episode/Time step 11/38. Reward 0.00
Episode/Time step 11/39. Reward 0.00
Episode/Time step 11/40. Reward 0.00
Episode/Time step 11/41. Reward 0.00
Episode/Time step 11/42. Reward 0.00
Episode/Time step 11/43. Reward -0.76
Episode/Time step 11/44. Reward -0.51
Episode/Time step 11/45. Reward 0.00
Episode/Time step 11/46. Reward -1.49
Episode/Time step 11/47. Reward -0.45
Episode/Time step 11/48. Reward -1.00
Episode/Time step 11/49. Reward -0.73
Episode/Time step 11/50. Reward -2.00
Episode/Time step 11/51. Reward -1.45
Episode/Time step 11/52. Reward -1.73
Episode/Time step 11/53. Reward -1.73
Episode/Time step 11/54. Reward -0.45
Episode/Time step 11/55. Reward -1.00
Episode/Time step 11/56. Reward -0.73
Episode/Time step 11/57.

Episode/Time step 25/31. Reward -0.72
Episode/Time step 25/32. Reward 0.00
Episode/Time step 25/33. Reward 0.00
Episode/Time step 25/34. Reward 0.00
Episode/Time step 25/35. Reward 0.00
Episode/Time step 25/36. Reward 0.00
Episode/Time step 25/37. Reward 0.00
Episode/Time step 25/38. Reward 0.00
Episode/Time step 25/39. Reward 0.00
Episode/Time step 25/40. Reward 0.00
Episode/Time step 25/41. Reward 0.00
Episode/Time step 25/42. Reward 0.00
Episode/Time step 25/43. Reward 0.00
Episode/Time step 25/44. Reward 0.00
Episode/Time step 25/45. Reward 0.00
Episode/Time step 25/46. Reward 0.00
Episode/Time step 25/47. Reward 0.00
Episode/Time step 25/48. Reward 0.00
Episode/Time step 25/49. Reward 0.00
Episode/Time step 25/50. Reward 0.00
Episode/Time step 25/51. Reward 0.00
Episode/Time step 25/52. Reward 0.00
Episode/Time step 25/53. Reward 0.00
Episode/Time step 25/54. Reward 0.00
Episode/Time step 25/55. Reward 0.00
Episode/Time step 25/56. Reward 0.00
Episode/Time step 25/57. Reward 0.00


In [173]:
rewards = np.array([np.average(mr_results_dict[i]['epi_rewards']) for i in range(1,21)])
filename = '../results/final/mean_reversion/mr_train'
np.save(filename, rewards)